In [1]:
import arcpy
import os

def join_feature_classes2(input_fc1, join_field_fc1, input_fc2, join_field_fc2, output_fc, unmatched_output=None):
    arcpy.env.overwriteOutput = True
    arcpy.env.qualifiedFieldNames = False
    
    # Perform the join
    joined = arcpy.management.AddJoin(input_fc1, join_field_fc1, input_fc2, join_field_fc2)
    
    # Copy the joined result to a new feature class
    arcpy.management.CopyFeatures(joined, output_fc)
    
    # Count the total number of records
    total_count = int(arcpy.management.GetCount(input_fc1)[0])
    
    # Use Statistics_analysis to quickly count non-null matches
    match_table = "memory\\match_table"
    arcpy.analysis.Statistics(output_fc, match_table, [[join_field_fc2, "COUNT"]], join_field_fc2)
    matched_count = int(arcpy.management.GetCount(match_table)[0])
    
    # Calculate the match percentage
    match_percentage = (matched_count / total_count) * 100
    
    print(f"Total records in {input_fc1}: {total_count}")
    print(f"Number of matched records: {matched_count}")
    print(f"Match percentage: {match_percentage:.2f}%")
    
    # If requested, save unmatched features
    if unmatched_output:
        # Get unmatched records using a selection query
        unmatched_fc = "in_memory\\unmatched_fc"
        arcpy.management.MakeFeatureLayer(output_fc, "lyr")
        arcpy.management.SelectLayerByAttribute("lyr", "NEW_SELECTION", f"{join_field_fc2} IS NULL")
        arcpy.management.CopyFeatures("lyr", unmatched_fc)
        
        # Save to the desired format
        desc = arcpy.Describe(input_fc2)
        if desc.dataType in ["FeatureClass", "Table"]:
            arcpy.management.CopyRows(unmatched_fc, unmatched_output)
        elif desc.dataType == "TextFile" or unmatched_output.endswith('.csv'):
            arcpy.TableToTable_conversion(unmatched_fc, os.path.dirname(unmatched_output), os.path.basename(unmatched_output))
        
        print(f"Unmatched features saved to {unmatched_output}")
    
    print("Function completed successfully.")



In [3]:
import datetime

join_feature_classes2(
    input_fc1=r"C:\data\DM_ALL_STRUCS.gdb\OH_DM_ALL_STRUCS_0805",
    join_field_fc1='SAP_FLOC_ID',
    input_fc2=r"C:\Users\mcveydb\Projects\local_problem_statements\Default.gdb\DR_IGPEP3Notifications",
    join_field_fc2='Floc',
    output_fc=fr"C:\Users\mcveydb\Projects\local_problem_statements\subset.gdb\output_feature_class_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}",
    unmatched_output=fr"C:\Users\mcveydb\Projects\local_problem_statements\subset.gdb\unmatched_output_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}",
)

